In [129]:
import pandas as pd
import numpy as np
from sklearn import svm

In [137]:
#definisco una funzione load_data che mi permette di generare un dataset da
#due normali multivariate (variabile x), mentre la variabile risposta è dicotomica 
#e può assumere valore [0,1].

def load_data(num_samples):

    #Definisco la variabile x1 
    x1 = np.random.multivariate_normal( [3,3], [[2,-0.5],[-0.5,2]], num_samples  )
    #Definisco la variabile x2
    x2 = np.random.multivariate_normal( [2,2], [[2,-0.5],[-0.5,2]], num_samples )
    #Definisco la variabile y [0,2]
    y1 = np.ones([num_samples]) *-1
    y2 = np.ones([num_samples]) *1
    #Concateno per avere il risultato finale
    x = np.concatenate([x1,x2], axis =0)
    y = np.concatenate([y1,y2], axis =0)
    
    #costruisco il dataset
    dataset1 = pd.DataFrame(x)
    dataset1.rename(columns={0:'x1',1:'x2'}, inplace=True)
    dataset2 = pd.DataFrame(y)
    dataset2.rename(columns={0:'y'}, inplace=True)
    dataset=pd.concat([dataset2,dataset1], axis=1)

    return dataset

In [138]:
np.random.seed(12345)
data=load_data(250)
data.head(10)

,y,x1,x2
0,-1.0,3.643647,3.185907
1,-1.0,3.099474,1.937973
2,-1.0,2.008915,6.404534
3,-1.0,3.140125,3.347873
4,-1.0,3.219651,4.939238
5,-1.0,0.751368,3.003512
6,-1.0,2.890794,3.505694
7,-1.0,2.255063,5.280277
8,-1.0,4.915873,0.440076
9,-1.0,0.754160,4.486214


In [139]:
#Definisco il modello svm con ottimizzazione GD nel caso generale

def svm_gd(x,y,w,b, max_iter = 10000, lr = 0.1, mylambda = 0.0001):
    ite = 0
    while True:
        # definisco la loss function:
        loss1 = mylambda /2 * np.linalg.norm( w )**2
        loss2 = np.mean( np.maximum(1- y*(np.matmul( x,w )+b), 0) )
        loss = loss1 + loss2 
        dLdw = mylambda*w -  np.mean( np.expand_dims( ((1 - y*(np.matmul( x,w )+b)) >0)*y, axis =-1) * x, axis = 0 , keepdims = False)
        dLdb = - np.mean(((1- y*(np.matmul( x,w )+b))> 0)*y, axis =0, keepdims = False)

        w -= lr * dLdw
        b -= lr * dLdb

        ite+=1
        if ite >= max_iter:
            break
    return w,b

In [140]:
#imposto le colonne del df come array
x=data[['x1','x2']].to_numpy()
y=data['y'].to_numpy()

In [141]:
#VERIFICO IL CORRETTO FUNZIONAMENTO DELL'ALGORITMO CON DUE INIZIALIZZAZIONI

#1
np.random.seed(1234)
w = np.random.normal(size=2) #[ 0.47143516, -1.19097569]
b = np.random.normal() #1.4327069684260973
w,b =svm_gd(x,y,w,b)
#w=array([-0.62212952, -0.5651318 ])
#b=3.00490696842606

#2
np.random.seed(54321)
w = np.random.normal(size=2) #[0.22397889, 0.7445909]
b = np.random.normal() #-0.3342689441687751
w,b =svm_gd(x,y,w,b)
#w=array([-0.62286001, -0.56563823])
#b=3.004731055831189

In [159]:
#IMPLEMENTAZIONE TRAMITE PACCHETTO PY
lin_clf = svm.LinearSVC()
mod=lin_clf.fit(x, y)
mod.coef_
#array([[-0.28070953, -0.27507687]])
mod.intercept_
#array([1.39832543])

/Users/mario/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [160]:
mod.loss

'squared_hinge'

In [144]:
lr=[0.1,0.01,0.001,0.0001]
diz={}
for i in lr:
    w,b =svm_gd(x,y,w,b,lr=0.001)
    w_lr=w.tolist()
    b_lr=b.tolist()
    diz[i]=(w_lr,b_lr)
diz

{0.1: ([-0.6220997067362539, -0.5656513326388231], 3.0055770558311194),
 0.01: ([-0.6220704129566184, -0.5659713884851937], 3.006325055831058),
 0.001: ([-0.6220516469359232, -0.5661148147446914], 3.0066710558310983),
 0.0001: ([-0.6220607813890746, -0.5661185955314554], 3.0066710558312257)}

In [169]:
def svm_gd_sto(x,y,w,b, max_iter = 10000, lr = 0.1, batch_size = 100, mylambda = 0.0001):
    ite = 0
    while True:
        batch_index = np.random.choice( x.shape[0], size=batch_size ,replace = False ) #estraggo 100 numeri casuali [0,500]
        x_batch = x[batch_index] #campione
        y_batch = y[batch_index] #campione

        # definisco la loss function:
        loss1 = mylambda /2 * np.linalg.norm( w )**2
        loss2 = np.mean( np.maximum(1- y_batch*(np.matmul( x_batch,w )+b), 0) )
        loss = loss1 + loss2 
        dLdw = mylambda*w -  np.mean( np.expand_dims( ((1 - y_batch*(np.matmul( x_batch,w )+b)) >0)*y_batch, axis =-1) * x_batch, axis = 0 , keepdims = False)
        dLdb = - np.mean(((1- y_batch*(np.matmul( x_batch,w )+b))> 0)*y_batch, axis =0, keepdims = False)

        w -= lr * dLdw
        b -= lr * dLdb

        ite+=1
        if ite >= max_iter:
            break
    return w,b

In [173]:
np.random.seed(1234)
w = np.random.normal(size=2) #[ 0.47143516, -1.19097569]
b = np.random.normal() #1.4327069684260973
w,b =svm_gd_sto(x,y,w,b)
#w=array([-0.6525253 , -0.59422346])
#b=3.0977069684260776